데이터 : 해양수산부 국립수산과학원

### 데이터불러오기

In [2]:
import requests
import pandas as pd
import json

In [3]:
url = 'http://apis.data.go.kr/1520635/OceanBiospeciesInfoService/getOceanBiospeciesResult?serviceKey=dXCD2cBMwTDD2W/hQd7doY5EYwJ82pKFPyaVU9i89OcBYenYBf4h9letUWJgAR5JO4FFgwzpjTMECuYtvPOtHA==&numOfRows=1000&pageNo=1'
params ={'serviceKey' : 'dXCD2cBMwTDD2W/hQd7doY5EYwJ82pKFPyaVU9i89OcBYenYBf4h9letUWJgAR5JO4FFgwzpjTMECuYtvPOtHA==',
        'numOfRows' : '10', 
        'pageNo' : '1' }

response = requests.get(url=url)
print(url,params)

http://apis.data.go.kr/1520635/OceanBiospeciesInfoService/getOceanBiospeciesResult?serviceKey=dXCD2cBMwTDD2W/hQd7doY5EYwJ82pKFPyaVU9i89OcBYenYBf4h9letUWJgAR5JO4FFgwzpjTMECuYtvPOtHA==&numOfRows=1000&pageNo=1 {'serviceKey': 'dXCD2cBMwTDD2W/hQd7doY5EYwJ82pKFPyaVU9i89OcBYenYBf4h9letUWJgAR5JO4FFgwzpjTMECuYtvPOtHA==', 'numOfRows': '10', 'pageNo': '1'}


### xml->dataframe

In [4]:
_df = pd.read_xml(response.text, xpath='.//item')

C:\Users\daega\AppData\Local\Temp\ipykernel_33984\2446481714.py:1: FutureWarning: Passing literal xml to 'read_xml' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  _df = pd.read_xml(response.text, xpath='.//item')


### 전처리

행 삭제 (필요한것만 남기기)

In [5]:
df = _df[["mfClassKor","mfDisinfo", "mfDistribution", "mfFamilyKor", "mfFeature", "mfSpeciesKor"]]
df.head()

,mfClassKor,mfDisinfo,mfDistribution,mfFamilyKor,mfFeature,mfSpeciesKor
0,조기어강,None,한국 남부 태평양과 인도양의 열대 및 온대 해역,돛새치과,좌우 새조골막은 하나로 융합되어 있으며 새파는 없다.양턱과 구개골에는 작지만 균일한...,청새치
1,조기어강,None,"북서태평양 (한국, 일본, 동중국해, 대만)",쥐치과,"몸은 타원형에 가까우며 매우 측편되어 있다. 체고는 높으며, 주둥이 끝은 뾰족하고...",쥐치
2,조기어강,None,"우리나라 남부, 일본 북해도 이남, 동중국해까지 분포한다.",쥐치과,몸은 타원형에 가까우며 옆으로 매우 납작하고 몸높이는 높다. 주둥이 끝은 뾰족하고 ...,쥐치
3,조기어강,None,None,쥐치과,None,쥐치
4,조기어강,전연안,"우리나라 전 연안, 일본 북해도이남, 동중국해, 대만",쥐치과,"몸은 타원형에 가깝고 측편되며, 체고가 높다. 주둥이는 뾰족하고 꼬리자루는 짧으며,...",쥐치


중복제거 & 결측치제거

In [6]:
df.drop_duplicates(subset='mfSpeciesKor',inplace=True)
df.dropna(subset='mfSpeciesKor',inplace=True)

C:\Users\daega\AppData\Local\Temp\ipykernel_33984\1596300023.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset='mfSpeciesKor',inplace=True)
C:\Users\daega\AppData\Local\Temp\ipykernel_33984\1596300023.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset='mfSpeciesKor',inplace=True)


우리나라 안 사는 어류종 삭제

In [7]:
# 한국 관련 아닌 키워드를 추출하여 조건을 생성
condition = (
    (df['mfDisinfo'] != '우리나라') & 
    (df['mfDisinfo'] != '한국') & 
    (df['mfDisinfo'] != '전연안') & 
    (~df['mfDistribution'].str.contains("우리나라", na=False)) & 
    (~df['mfDistribution'].str.contains("한국", na=False)) &
    (~df['mfDistribution'].str.contains("동해", na=False)) &
    (~df['mfDistribution'].str.contains("서해", na=False)) & 
    (~df['mfDistribution'].str.contains("남해", na=False)) & 
    (~df['mfDistribution'].str.contains("제주", na=False)) & 
    (~df['mfDistribution'].str.contains("울릉", na=False)) &
    (~df['mfDistribution'].str.contains("독도", na=False))
)

# 조건에 맞는 행 삭제
df = df[~condition]

고래 & 상어 & 해파리 삭제

In [8]:
condition = (
    (df['mfSpeciesKor'].str.contains("해파리", na=False)) |
    (df['mfSpeciesKor'].str.contains("상어", na=False)) |
    (df['mfSpeciesKor'].str.contains("고래", na=False)) |
    (df['mfFamilyKor'].str.contains("해파리", na=False)) |
    (df['mfFamilyKor'].str.contains("상어", na=False)) |
    (df['mfFamilyKor'].str.contains("고래", na=False)) 
)

df = df[~condition]

지역별로 나누기

In [9]:
df['지역'] = None
df.head()

,mfClassKor,mfDisinfo,mfDistribution,mfFamilyKor,mfFeature,mfSpeciesKor,지역
0,조기어강,None,한국 남부 태평양과 인도양의 열대 및 온대 해역,돛새치과,좌우 새조골막은 하나로 융합되어 있으며 새파는 없다.양턱과 구개골에는 작지만 균일한...,청새치,None
1,조기어강,None,"북서태평양 (한국, 일본, 동중국해, 대만)",쥐치과,"몸은 타원형에 가까우며 매우 측편되어 있다. 체고는 높으며, 주둥이 끝은 뾰족하고...",쥐치,None
7,이형세대강,"제주도,완도,독도","일본, 오스트레일리아, 인도양에 분포하고 우리나라에서는 제주도, 독도, 완도에 분포한다.",스포로크누스과,뿌리는 상당히 큰 원추상반이며 황갈색의 털로 덮였다. 털은 뿌리의 상당히 상부에까지...,스포로크누스,None
8,복족강,None,"한국, 일본(홋카이도, 혼슈, 시코쿠, 큐우슈우, 초시)",밤고둥과,"패각은 난원형, 나탑은 낮은 원추형, 나층은 7층, 각 층은 팽윤되어 있다.\r\r...",개울타리고둥,None
9,None,"주문진,속초,부산,비진도,거제도,욕지도,완도,여수,어청도,제주도","일본에 분포되고 우리나라에서는 주문진, 속초, 부산, 비진도, 거제도, 욕지도, 완...",모자반과,몸의 높이는 2∼3m 이며 대체로 전기한 종과 매우 비슷하다. 그리고 계절과 산지에...,꽈배기모자반,None


In [10]:
### 우선순위 : 서해-> 울릉 -> 제주 -> 부산 -> 동해 -> 남해
## 우선순위대로 하기 위해 반대로 코딩하면 됨
## 서해 어류종이 너무 적어 1순위 (마지막에 count)

ulleung = '울릉도'
jeju = '제주'
busan = '부산'
east = '동해'
south = '남해'
west = '서해'

In [11]:
# 남해

condition = (
    (df['mfDisinfo'].str.contains(south, na=False)) |
    (df['mfDistribution'].str.contains(south, na=False))
)

df.loc[condition, ['지역']] = south

print(south , len(df[df['지역']==south]))

남해 62


In [12]:
# 동해

condition = (
    (df['mfDisinfo'].str.contains(east, na=False)) |
    (df['mfDistribution'].str.contains(east, na=False))
)

df.loc[condition, ['지역']] = east

print(east , len(df[df['지역']==east]))

동해 66


In [13]:
# 부산

condition = (
    (df['mfDisinfo'].str.contains(busan, na=False)) |
    (df['mfDistribution'].str.contains(busan, na=False))
)

df.loc[condition, ['지역']] = busan

print(busan , len(df[df['지역']==busan]))

부산 52


In [14]:
# 제주도

condition = (
    (df['mfDisinfo'].str.contains(jeju, na=False)) |
    (df['mfDistribution'].str.contains(jeju, na=False))
)

df.loc[condition, ['지역']] = jeju

print(jeju , len(df[df['지역']==jeju]))

제주 64


In [15]:
# 울릉도

condition = (
    (df['mfDisinfo'].str.contains(ulleung, na=False)) |
    (df['mfDistribution'].str.contains(ulleung, na=False))
)

df.loc[condition, ['지역']] = ulleung

print(ulleung , len(df[df['지역']==ulleung]))

울릉도 24


In [16]:
# 서해

condition = (
    (df['mfDisinfo'].str.contains(west, na=False)) |
    (df['mfDistribution'].str.contains(west, na=False))
)

df.loc[condition, ['지역']] = west

print(west , len(df[df['지역']==west]))

서해 20


지역별 개수 확인

In [17]:
value_counts = df['지역'].value_counts()
print(value_counts)

지역
제주     53
동해     42
남해     25
울릉도    23
부산     21
서해     20
Name: count, dtype: int64


### 20개씩만 남기기

In [18]:
group = df.groupby('지역').head(20)
df = group.sort_values(by='지역')

df

,mfClassKor,mfDisinfo,mfDistribution,mfFamilyKor,mfFeature,mfSpeciesKor,지역
266,연갑강,남해안,우리나라 남해안이 주 서식지이며 특히 거문도 인근해역에 다량으로 분포한다.,도화새우과,이마뿔은 길고 위로 많이 휘어 있다. 윗가장자리에는 12~15개의 이가 있으며 뒤의...,긴줄꼬마도화새우,남해
636,연골어강,None,"태평양 서부(한국 서, 남해, 일본 남부, 동중국해, 남중국해, 피지)",색가오리과,"체반의 가장자리는 일직선 형태로, 가슴지느러미 모서리는 비교적 둥글지만 폭이 좁고 ...",노랑가오리,남해
768,조기어강,None,"우리나라의 남해, 일본 연안, 대만까지 분포한다.",보리멸과,None,청보리멸,남해
176,조기어강,None,"태평양 서부 (한국 남해, 일본, 중국, 대만), 인도양 (인도, 아프리카 남부)",홍갈치과,"체형은 매우 길고, 몸의 뒤쪽으로 갈수록 가늘어진다. 주둥이는 짧고 뭉툭하며, 눈은...",남방홍갈치,남해
856,조기어강,None,"북서태평양 (한국 남해, 일본 남부, 동중국해, 대만)",성대과,"몸은 횡단면이 둥글고, 체고는 등지느러미 기부에서 가장 높다. 눈은 크며 머리 등...",히메성대,남해
...,...,...,...,...,...,...,...
319,None,None,"한국(제주도), 일본, 태평양",None,"부채꼴, 난원형, 원반형",풀크라관이끼벌레,제주
320,None,None,"한국(제주도), 태평양",None,군체는 두 갈래로 가지를 치며 가지는 관절을 가지지 않고 서로 엉킨 모습을 나타낸다...,볼록이은이끼벌레,제주
321,나후강,None,"한국(제주도), 태평양, 인도양",산호붙이이끼벌레과,"군체는 기립형, 가지를 치며 가지는 딱딱하므로 쉽게 부러진다.",넓적부리이끼벌레,제주
125,이형세대,"구룡포,부산,비진도,거문도,거제도,욕지도,흑산도,제주도","일본에 분포되고, 우리나라에는 함경도, 구룡포, 부산, 목도, 비진도, 거문도, 거...",빨간검둥이과,"뿌리는 사상, 몸은 연골질이며 높이 20cm 혹은 그 이상, 원주상이고 기부의 굵기...",오카무라서실,제주


### 저장, 조회

In [19]:
df[df['지역'] == '남해']

,mfClassKor,mfDisinfo,mfDistribution,mfFamilyKor,mfFeature,mfSpeciesKor,지역
266,연갑강,남해안,우리나라 남해안이 주 서식지이며 특히 거문도 인근해역에 다량으로 분포한다.,도화새우과,이마뿔은 길고 위로 많이 휘어 있다. 윗가장자리에는 12~15개의 이가 있으며 뒤의...,긴줄꼬마도화새우,남해
636,연골어강,None,"태평양 서부(한국 서, 남해, 일본 남부, 동중국해, 남중국해, 피지)",색가오리과,"체반의 가장자리는 일직선 형태로, 가슴지느러미 모서리는 비교적 둥글지만 폭이 좁고 ...",노랑가오리,남해
768,조기어강,None,"우리나라의 남해, 일본 연안, 대만까지 분포한다.",보리멸과,None,청보리멸,남해
176,조기어강,None,"태평양 서부 (한국 남해, 일본, 중국, 대만), 인도양 (인도, 아프리카 남부)",홍갈치과,"체형은 매우 길고, 몸의 뒤쪽으로 갈수록 가늘어진다. 주둥이는 짧고 뭉툭하며, 눈은...",남방홍갈치,남해
856,조기어강,None,"북서태평양 (한국 남해, 일본 남부, 동중국해, 대만)",성대과,"몸은 횡단면이 둥글고, 체고는 등지느러미 기부에서 가장 높다. 눈은 크며 머리 등...",히메성대,남해
504,조기어강,남해안,"우리나라 남해, 일본 중부이남, 동중국해, 대만, 필리핀",나비고기과,"몸은 원통형에 가깝고, 체고가 높고 매우 측편하며, 머리는 짧은 편이다. 주둥이는 ...",세동가리돔,남해
857,조기어강,남해안,"우리나라 남해, 일본 남부해, 동중국해, 남중국해, 인도양",성대과,"몸은 가늘고 길며, 머리는 납작하다. 주둥이 앞쪽은 중앙이 약간 오목하고, 그 양쪽...",가시달강어,남해
190,조기어강,남해안,"우리나라 남해, 일본 남부해, 황해, 동중국해",민어과,"몸은 약간 길고 측편하며, 눈은 큰 편이다. 주둥이는 둥글고 둔하며, 그 길이는 눈...",흑조기,남해
858,조기어강,None,"북서태평양 (한국 남해, 일본 남부, 동중국해)",성대과,몸은 가늘며 횡단면이 원통형이다. 주둥이 앞끝은 돌출되어 있으며 가시는 짧다. 가슴...,달강어,남해
376,조기어강,None,"북태평양 (한국 동, 남해, 일본, 러시아, 알래스카, 미국, 멕시코), 북극",청어과,"몸은 다소 측편되어 있으며, 눈 주위로 기름눈까풀이 나타난다. 입은 경사지고 위턱...",청어,남해
